<a href="https://colab.research.google.com/github/Gustavo0501/Trabalho_Grafos/blob/main/Etapa3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integrantes:
# Alexandre Marques Spinola
# Gustavo do Carmo Resende




In [ ]:
import os
import math
import sys
import copy
import time

class Rota:
    """Representa a rota de um único veículo, com seus custos e serviços."""

    def __init__(self, id_rota, deposito):
        self.id_rota = id_rota
        self.deposito = deposito
        self.carga_total = 0
        self.custo_total = 0
        self.servicos_atendidos = []
        self.posicao_atual = deposito
        self.sequencia_visitas = [f"(D 0,{deposito},{deposito})"] # Toda rota inicia no depósito

    def adicionar_servico(self, servico, custo_deslocamento, ponto_entrada, ponto_saida):
        """Adiciona um serviço à rota e atualiza os totais de custo e carga."""
        self.carga_total += servico['demanda']
        self.custo_total += custo_deslocamento + servico['custo']
        self.posicao_atual = ponto_saida
        self.servicos_atendidos.append(servico)
        self.sequencia_visitas.append(f"(S {servico['id']},{ponto_entrada},{ponto_saida})")

    def finalizar_rota(self, distancias):
        """Calcula o custo de retorno ao depósito e completa a sequência de visitas."""
        custo_retorno = distancias[self.posicao_atual][self.deposito]
        self.custo_total += custo_retorno
        self.sequencia_visitas.append(f"(D 0,{self.deposito},{self.deposito})")

    def to_string(self):
        """Formata a rota como uma string para o arquivo de solução."""
        return f" 0 1 {self.id_rota} {self.carga_total} {int(self.custo_total)} {len(self.sequencia_visitas)} {' '.join(self.sequencia_visitas)}"

class Solucao:
    """Armazena o conjunto de todas as rotas que compõem a solução final."""

    def __init__(self):
        self.rotas = []
        self.custo_total = 0

    def adicionar_rota(self, rota):
        """Adiciona uma rota completa à solução."""
        self.rotas.append(rota)
        self.custo_total += rota.custo_total

    def salvar_em_arquivo(self, nome_arquivo_instancia, pasta_saida, tempo_total_ns, tempo_encontrar_ns):
        """Salva a solução completa em um arquivo, seguindo o formato padrão."""
        os.makedirs(pasta_saida, exist_ok=True)
        caminho_saida = os.path.join(pasta_saida, f"sol-{os.path.splitext(nome_arquivo_instancia)[0]}.dat")

        with open(caminho_saida, 'w') as f:
            custo_final = sum(r.custo_total for r in self.rotas)
            f.write(f"{int(custo_final)}\n")
            f.write(f"{len(self.rotas)}\n")

            # Escreve os dois valores de tempo medidos em nanossegundos
            f.write(f"{tempo_total_ns}\n")      # Tempo total de execução (Etapa 2 + Etapa 3)
            f.write(f"{tempo_encontrar_ns}\n")  # Tempo para encontrar a primeira solução (só Etapa 2)

            for rota in self.rotas:
                f.write(f"{rota.to_string()}\n")
        print(f"Solução salva em: {caminho_saida}")

class Grafo:
    """Lê, armazena e pré-processa todos os dados de uma instância."""

    def __init__(self, caminho_arquivo):
        self.nome_arquivo = os.path.basename(caminho_arquivo)
        self.vertices = 0
        self.deposito = None
        self.capacidade = 0
        self.max_veiculos = -1
        self.arestas = []
        self.arcos = []
        self.servicos_requeridos = []
        self.distancias = []
        self.predecessores = []

        self._ler_arquivo_instancia(caminho_arquivo)
        self._calcular_caminhos_curtos()

    def _ler_arquivo_instancia(self, caminho_arquivo):
        """Lê o arquivo de instância de forma robusta, preenchendo os atributos do grafo."""
        with open(caminho_arquivo, 'r') as f:
            # Filtro inicial para remover linhas em branco ou de rodapé
            linhas = [linha.strip() for linha in f if linha.strip() and not linha.lower().startswith("based")]

        secao = None
        id_servico_counter = 1

        for linha in linhas:
            if linha == "-1":
                continue

            # Leitura do cabeçalho da instância
            if ':' in linha:
                chave, valor = [p.strip() for p in linha.split(':')]
                if chave == 'Depot Node': self.deposito = int(valor)
                elif chave == '#Nodes': self.vertices = int(valor)
                elif chave == 'Capacity': self.capacidade = int(valor)
                elif chave == '#Vehicles': self.max_veiculos = int(valor)
            # Identificação da seção atual de dados
            elif linha.startswith('ReN.'): secao = 'ReN'
            elif linha.startswith('ReE.'): secao = 'ReE'
            elif linha.startswith('ReA.'): secao = 'ReA'
            elif linha.startswith('EDGE'): secao = 'EDGE'
            elif linha.startswith('ARC'): secao = 'ARC'
            else:
                partes = linha.split()
                if not partes: continue

                # Bloco para tratar a leitura dos dados de cada seção
                try:
                    if secao == 'ReN':
                        no = int(partes[0][1:])
                        demanda, custo_servico = int(partes[1]), int(partes[2])
                        self.servicos_requeridos.append({'id': id_servico_counter, 'tipo': 'V', 'nos': (no, no), 'custo': custo_servico, 'demanda': demanda})
                        id_servico_counter += 1
                    elif secao == 'ReE':
                        u, v = int(partes[1]), int(partes[2])
                        custo_servico, demanda = int(partes[3]), int(partes[4])
                        self.servicos_requeridos.append({'id': id_servico_counter, 'tipo': 'E', 'nos': (u, v), 'custo': custo_servico, 'demanda': demanda})
                        self.arestas.append((u, v, custo_servico))
                        id_servico_counter += 1
                    elif secao == 'ReA':
                        u, v = int(partes[1]), int(partes[2])
                        custo_servico, demanda = int(partes[3]), int(partes[4])
                        self.servicos_requeridos.append({'id': id_servico_counter, 'tipo': 'A', 'nos': (u, v), 'custo': custo_servico, 'demanda': demanda})
                        self.arcos.append((u, v, custo_servico))
                        id_servico_counter += 1
                    elif secao == 'EDGE':
                        u, v, custo = int(partes[1]), int(partes[2]), int(partes[3])
                        self.arestas.append((u, v, custo))
                    elif secao == 'ARC':
                        u, v, custo = int(partes[1]), int(partes[2]), int(partes[3])
                        self.arcos.append((u, v, custo))
                except (ValueError, IndexError):
                    # Ignora linhas malformadas ou cabeçalhos de seções (ex: "FROM N. TO N...")
                    continue

    def _calcular_caminhos_curtos(self):
        """Executa o algoritmo de Floyd-Warshall para preencher a matriz de distâncias."""
        V = self.vertices
        self.distancias = [[math.inf] * (V + 1) for _ in range(V + 1)]
        self.predecessores = [[-1] * (V + 1) for _ in range(V + 1)]

        for i in range(1, V + 1):
            self.distancias[i][i] = 0

        for u, v, custo in self.arestas:
            if custo < self.distancias[u][v]:
                self.distancias[u][v] = self.distancias[v][u] = custo
                self.predecessores[u][v] = u
                self.predecessores[v][u] = v

        for u, v, custo in self.arcos:
            if custo < self.distancias[u][v]:
                self.distancias[u][v] = custo
                self.predecessores[u][v] = u

        # Algoritmo de Floyd-Warshall
        for k in range(1, V + 1):
            for i in range(1, V + 1):
                for j in range(1, V + 1):
                    if self.distancias[i][k] + self.distancias[k][j] < self.distancias[i][j]:
                        self.distancias[i][j] = self.distancias[i][k] + self.distancias[k][j]
                        self.predecessores[i][j] = self.predecessores[k][j]
        print(f"Matriz de distâncias calculada para {self.nome_arquivo}.")

class PathScanning:
    """(Etapa 2) Implementa a heurística construtiva Path Scanning."""

    def __init__(self, grafo):
        self.grafo = grafo

    def executar(self):
        """Executa o algoritmo e retorna um objeto Solucao, respeitando o limite de veículos."""
        solucao = Solucao()
        servicos_nao_atendidos = set(s['id'] for s in self.grafo.servicos_requeridos)
        servicos_map = {s['id']: s for s in self.grafo.servicos_requeridos}
        rota_id = 1

        limite_veiculos = self.grafo.max_veiculos if self.grafo.max_veiculos > 0 else float('inf')

        while servicos_nao_atendidos:
            # Verifica a restrição de veículos antes de criar uma nova rota
            if rota_id > limite_veiculos:
                print(f"AVISO: Limite de {int(limite_veiculos)} veículos atingido, mas ainda há {len(servicos_nao_atendidos)} serviços não atendidos.")
                print("       A heurística não conseguiu encontrar uma solução viável com a frota disponível.")
                break

            rota_atual = Rota(rota_id, self.grafo.deposito)

            # Constrói uma rota adicionando o serviço mais próximo sucessivamente
            while True:
                candidatos = []
                for sid in servicos_nao_atendidos:
                    servico = servicos_map[sid]
                    if rota_atual.carga_total + servico['demanda'] <= self.grafo.capacidade:
                        u, v = servico['nos']
                        dist_para_u = self.grafo.distancias[rota_atual.posicao_atual][u]
                        dist_para_v = self.grafo.distancias[rota_atual.posicao_atual][v]

                        # Para arestas (não direcionadas), escolhe o ponto de entrada mais próximo
                        if servico['tipo'] == 'E' and dist_para_v < dist_para_u:
                            custo_deslocamento = dist_para_v
                            ponto_entrada, ponto_saida = v, u
                        else:
                            custo_deslocamento = dist_para_u
                            ponto_entrada, ponto_saida = u, v

                        candidatos.append((custo_deslocamento, servico, ponto_entrada, ponto_saida))

                if not candidatos:
                    break

                # Escolha gulosa: seleciona o candidato com menor custo de deslocamento
                candidatos.sort(key=lambda x: x[0])
                melhor_custo, melhor_servico, entrada, saida = candidatos[0]

                rota_atual.adicionar_servico(melhor_servico, melhor_custo, entrada, saida)
                servicos_nao_atendidos.remove(melhor_servico['id'])

            rota_atual.finalizar_rota(self.grafo.distancias)
            solucao.adicionar_rota(rota_atual)
            rota_id += 1

        return solucao

class BuscaLocal:
    """(Etapa 3) Aplica algoritmos de busca local para refinar uma solução existente."""

    def __init__(self, solucao_inicial, grafo):
        self.solucao_atual = copy.deepcopy(solucao_inicial)
        self.grafo = grafo
        self.servicos_map = {s['id']: s for s in self.grafo.servicos_requeridos}

    def executar(self):
        """Executa o processo de busca local até que nenhuma melhoria seja encontrada."""
        print("\n--- Iniciando Busca Local para refinar a solução ---")
        melhoria_encontrada = True
        iteracao = 0
        while melhoria_encontrada:
            iteracao += 1
            print(f"Iteração {iteracao} da Busca Local... Custo atual: {int(self.solucao_atual.custo_total)}")

            # Tenta aplicar os movimentos de vizinhança
            melhoria_encontrada = self._tentar_transferencia()
            if not melhoria_encontrada:
                melhoria_encontrada = self._tentar_re_insercao_intra_rota()

        print(f"Busca Local finalizada. Custo final: {int(self.solucao_atual.custo_total)}")
        return self.solucao_atual

    def _recalcular_custo_e_sequencia_rota(self, lista_servicos):
        """Função auxiliar para recalcular o custo e a sequência de uma rota modificada."""
        custo = 0
        posicao_atual = self.grafo.deposito
        sequencia_visitas = [f"(D 0,{self.grafo.deposito},{self.grafo.deposito})"]

        for servico in lista_servicos:
            ponto_entrada, ponto_saida = servico['nos']
            custo_deslocamento = self.grafo.distancias[posicao_atual][ponto_entrada]
            custo += custo_deslocamento + servico['custo']
            posicao_atual = ponto_saida
            sequencia_visitas.append(f"(S {servico['id']},{ponto_entrada},{ponto_saida})")

        custo += self.grafo.distancias[posicao_atual][self.grafo.deposito]
        sequencia_visitas.append(f"(D 0,{self.grafo.deposito},{self.grafo.deposito})")

        return custo, sequencia_visitas

    def _tentar_transferencia(self):
        """Tenta mover um serviço de uma rota para outra (movimento Relocate inter-rotas)."""
        for rota_origem in self.solucao_atual.rotas:
            for i in range(len(rota_origem.servicos_atendidos) - 1, -1, -1):
                servico_a_mover = rota_origem.servicos_atendidos[i]

                for rota_destino in self.solucao_atual.rotas:
                    if rota_origem.id_rota == rota_destino.id_rota:
                        continue

                    # Verifica a restrição de capacidade antes de testar a inserção
                    if rota_destino.carga_total + servico_a_mover['demanda'] > self.grafo.capacidade:
                        continue

                    custo_antigo_combinado = rota_origem.custo_total + rota_destino.custo_total

                    # Simula a remoção da rota de origem e encontra a melhor inserção na de destino
                    servicos_origem_temp = rota_origem.servicos_atendidos[:i] + rota_origem.servicos_atendidos[i+1:]
                    novo_custo_origem, _ = self._recalcular_custo_e_sequencia_rota(servicos_origem_temp)
                    melhor_custo_destino = float('inf')
                    melhor_posicao_insercao = -1

                    for j in range(len(rota_destino.servicos_atendidos) + 1):
                        servicos_destino_temp = rota_destino.servicos_atendidos[:j] + [servico_a_mover] + rota_destino.servicos_atendidos[j:]
                        custo_teste, _ = self._recalcular_custo_e_sequencia_rota(servicos_destino_temp)
                        if custo_teste < melhor_custo_destino:
                            melhor_custo_destino = custo_teste
                            melhor_posicao_insercao = j

                    # Se o custo total combinado diminuir, o movimento é aceito
                    if novo_custo_origem + melhor_custo_destino < custo_antigo_combinado:
                        # Efetiva a mudança (first improvement)
                        rota_origem.servicos_atendidos.pop(i)
                        rota_origem.carga_total -= servico_a_mover['demanda']
                        rota_origem.custo_total, rota_origem.sequencia_visitas = self._recalcular_custo_e_sequencia_rota(rota_origem.servicos_atendidos)

                        rota_destino.servicos_atendidos.insert(melhor_posicao_insercao, servico_a_mover)
                        rota_destino.carga_total += servico_a_mover['demanda']
                        rota_destino.custo_total, rota_destino.sequencia_visitas = self._recalcular_custo_e_sequencia_rota(rota_destino.servicos_atendidos)

                        self.solucao_atual.custo_total = sum(r.custo_total for r in self.solucao_atual.rotas)

                        print(f"  Melhoria (Transferência): Serviço {servico_a_mover['id']} movido da Rota {rota_origem.id_rota} para Rota {rota_destino.id_rota}.")
                        return True # Encerra e reinicia o processo de busca
        return False

    def _tentar_re_insercao_intra_rota(self):
        """Tenta mover um serviço para uma posição diferente dentro da mesma rota."""
        for rota in self.solucao_atual.rotas:
            if len(rota.servicos_atendidos) < 2:
                continue

            for i in range(len(rota.servicos_atendidos)):
                servico_a_mover = rota.servicos_atendidos[i]

                servicos_temp = rota.servicos_atendidos[:i] + rota.servicos_atendidos[i+1:]
                custo_original = rota.custo_total

                for j in range(len(servicos_temp) + 1):
                    if i == j: continue

                    servicos_teste = servicos_temp[:j] + [servico_a_mover] + servicos_temp[j:]
                    novo_custo, _ = self._recalcular_custo_e_sequencia_rota(servicos_teste)

                    if novo_custo < custo_original:
                        # Efetiva a mudança (first improvement)
                        rota.servicos_atendidos = servicos_teste
                        rota.custo_total, rota.sequencia_visitas = self._recalcular_custo_e_sequencia_rota(rota.servicos_atendidos)
                        self.solucao_atual.custo_total = sum(r.custo_total for r in self.solucao_atual.rotas)

                        print(f"  Melhoria (Re-inserção): Serviço {servico_a_mover['id']} reposicionado na Rota {rota.id_rota}.")
                        return True
        return False

def main():
    """Função principal para executar o processo completo para todas as instâncias."""
    pasta_entrada = 'MCGRP/'
    pasta_saida = 'G13/'

    if not os.path.isdir(pasta_entrada):
        print(f"Erro: A pasta de entrada '{pasta_entrada}' não foi encontrada.")
        sys.exit(1)

    arquivos_instancia = [f for f in os.listdir(pasta_entrada) if f.endswith('.dat')]

    for nome_arquivo in arquivos_instancia:
        print(f"\n--- Processando instância: {nome_arquivo} ---")
        try:
            # Medição de tempo: Início do processo
            inicio_processo_ns = time.perf_counter_ns()

            caminho_completo = os.path.join(pasta_entrada, nome_arquivo)

            # Etapa 1: Leitura e pré-processamento
            grafo = Grafo(caminho_completo)

            # Etapa 2: Geração da Solução Inicial
            algoritmo_construtivo = PathScanning(grafo)
            solucao_inicial = algoritmo_construtivo.executar()

            # Medição de tempo: Fim da Etapa 2
            fim_etapa2_ns = time.perf_counter_ns()
            tempo_para_encontrar_ns = fim_etapa2_ns - inicio_processo_ns

            # Verifica se a solução inicial é factível (cobriu todos os serviços)
            servicos_atendidos_count = sum(len(r.servicos_atendidos) for r in solucao_inicial.rotas)
            if servicos_atendidos_count < len(grafo.servicos_requeridos):
                print("A solução inicial é infactível (não cobre todos os serviços devido ao limite de veículos).")
                print("Salvando a solução parcial gerada pela Etapa 2...")
                solucao_final = solucao_inicial
            else:
                print(f"Custo da solução inicial (Path Scanning): {int(solucao_inicial.custo_total)}")
                # Etapa 3: Refinamento com Busca Local
                busca_local = BuscaLocal(solucao_inicial, grafo)
                solucao_melhorada = busca_local.executar()
                solucao_final = solucao_melhorada

            # Medição de tempo: Fim de todo o processo
            fim_total_ns = time.perf_counter_ns()
            tempo_total_ns = fim_total_ns - inicio_processo_ns

            # Salva a solução final, passando os dois tempos medidos
            solucao_final.salvar_em_arquivo(nome_arquivo, pasta_saida, tempo_total_ns, tempo_para_encontrar_ns)

        except Exception as e:
            print(f"Ocorreu um erro ao processar {nome_arquivo}: {e}")
            import traceback
            traceback.print_exc()

if __name__ == "__main__":
    main()


# Readme abaixo

In [ ]:
# Projeto de Algoritmos em Grafos - Roteamento de Veículos (MCGRP)

**Universidade Federal de Lavras (UFLA)** **Disciplina:** GCC262 - Grafos e suas Aplicações
**Professor:** Mayron César O. Moreira

**Integrantes da dupla:** Alexandre Marques Spinola, Gustavo do Carmo Resende

---

## 1. Introdução

Este projeto visa desenvolver uma solução computacional para uma variação do Problema do Carteiro Chinês Misto (Mixed Chinese Postman Problem - MCGRP), um desafio clássico na área de otimização e logística. O objetivo é determinar um conjunto de rotas de custo mínimo para uma frota de veículos, partindo de um depósito, para atender a uma série de serviços requeridos (em nós, arestas e arcos de um multigrafo), respeitando a capacidade de cada veículo.

A solução foi desenvolvida em Python 3 e segue uma abordagem em duas fases principais, conforme especificado no trabalho prático:
* **Etapa 2:** Geração de uma solução inicial através de uma heurística construtiva.
* **Etapa 3:** Aprimoramento da solução inicial por meio de um algoritmo de busca local.

## 2. Estrutura do Projeto

O código foi modularizado em classes para garantir clareza, manutenibilidade e desacoplamento lógico, facilitando o entendimento e a extensão do projeto.

* `main.py`: Arquivo principal que orquestra todo o processo de execução, desde a leitura das instâncias até o salvamento das soluções finais.

### Classes Principais:

* **`Grafo`**: Responsável por toda a interação com os dados de entrada. Esta classe lê um arquivo de instância `.dat`, armazena todas as suas informações (capacidade, depósito, serviços, etc.) e realiza o pré-processamento necessário, como o cálculo da matriz de distâncias mínimas entre todos os nós utilizando o algoritmo de **Floyd-Warshall**.

* **`Rota`**: Modela a rota de um único veículo. Controla a sequência de visitas, o custo total da rota e a carga acumulada, garantindo que as restrições sejam respeitadas em nível individual.

* **`Solucao`**: Representa a solução completa para uma instância, contendo um conjunto de objetos `Rota`. É responsável por calcular o custo total e formatar a saída para o arquivo `.dat` final.

* **`PathScanning` (Etapa 2)**: Implementa a heurística construtiva para gerar a solução inicial. O algoritmo opera de forma gulosa, iniciando uma nova rota e adicionando iterativamente o serviço mais próximo da posição atual do veículo, até que nenhum serviço possa mais ser adicionado sem violar a capacidade.

* **`BuscaLocal` (Etapa 3)**: Implementa o algoritmo de melhoria. Partindo da solução gerada pelo `PathScanning`, ele busca iterativamente por otimizações, aplicando movimentos de vizinhança para reduzir o custo total.

## 3. Algoritmos Implementados

### Etapa 2: Heurística Construtiva - Path Scanning

A solução inicial é construída de forma sequencial e gulosa:
1.  Uma nova rota é iniciada.
2.  Enquanto houver capacidade disponível no veículo, o algoritmo "escaneia" todos os serviços ainda não atendidos e seleciona aquele cujo ponto de acesso está mais próximo da posição atual do veículo.
3.  O serviço selecionado é adicionado à rota.
4.  O processo se repete até que nenhum serviço restante possa ser adicionado à rota atual. A rota é então finalizada (retornando ao depósito) e uma nova rota é iniciada, se necessário.
5.  O algoritmo também respeita a restrição de número de veículos, caso seja especificada na instância.

### Etapa 3: Busca Local

Para refinar a solução inicial, uma busca local do tipo *Variable Neighborhood Descent (VND)* simplificada é aplicada, utilizando a estratégia de *first improvement*. Os seguintes movimentos de vizinhança são explorados em sequência:

1.  **Transferência (Relocate Inter-Rotas):** Tenta mover um serviço de uma rota para todas as posições possíveis em outra rota. Se uma transferência resultar em uma redução do custo total da solução, o movimento é efetivado e a busca é reiniciada.
2.  **Re-inserção (Relocate Intra-Rota):** Caso nenhuma transferência melhore a solução, o algoritmo tenta alterar a posição de um serviço dentro de sua própria rota. Se uma posição melhor for encontrada, o movimento é efetivado e a busca é reiniciada.

O processo termina quando nenhum dos movimentos consegue mais aprimorar o custo da solução.

## 4. Como Executar

O projeto foi desenvolvido para ser executado de forma simples, processando todas as instâncias de uma pasta e salvando os resultados em outra.

### Pré-requisitos
* Python 3.x

### Passos para Execução
1.  Clone ou baixe este repositório.
2.  Crie uma pasta chamada `MCGRP` no mesmo diretório do script e coloque todos os arquivos de instância (`.dat`) dentro dela.
3.  Crie uma pasta de saída. No código, ela está definida como `G13`, mas você pode alterar o nome na função `main`.
    ```python
    # Em main()
    pasta_entrada = 'MCGRP/'
    pasta_saida = 'G13/'
    ```
4.  Execute o script principal a partir do seu terminal:
    ```bash
    python nome_do_arquivo.py
    ```
    O script irá processar cada arquivo da pasta `MCGRP` e salvar a solução otimizada correspondente na pasta `G13`, seguindo o formato `sol-nome_da_instancia.dat`.

## 5. Formato da Saída

Cada arquivo de solução gerado contém:
1.  **Linha 1:** O custo total da solução final.
2.  **Linha 2:** O número total de rotas utilizadas.
3.  **Linha 3:** O tempo total de execução do algoritmo em nanossegundos (Etapa 2 + Etapa 3).
4.  **Linha 4:** O tempo para encontrar a primeira solução factível em nanossegundos (apenas Etapa 2).
5.  **Linhas seguintes:** A descrição de cada rota, conforme o padrão especificado no trabalho.

